# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy import stats as st 
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

In [2]:
df_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
df_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
df_3 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

In [3]:
df_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
df_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
df_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [8]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Данные содержать информацию о трех признаках и одним целевом признаке для каждого региона.  
Пропуски отсутсвуют, тип данных указан верно. Для каждого региона представлена информация о 100 000 скважин.

## Обучение и проверка модели

In [9]:
features_1 = df_1.drop(columns = ['product', 'id'], axis = 1)
target_1 = df_1['product']
features_2 = df_2.drop(columns = ['product', 'id'], axis = 1)
target_2 = df_2['product']
features_3 = df_3.drop(columns = ['product', 'id'], axis = 1)
target_3 = df_3['product']

Разделили целевой признак и объясняющие для каждого региона.

In [10]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(
    features_3, target_3, test_size=0.25, random_state=12345)

Исходные данные разделены на обучающую и валидационную выборки в соотношении 75:25.

In [11]:
def regression (features_train, target_train, features_valid, target_valid):
    lr = LinearRegression()
    lr.fit(features_train, target_train)
    predictions_valid = lr.predict(features_valid)
    rmse = mean_squared_error(target_valid, predictions_valid)**0.5
    mae = mean_absolute_error(target_valid, predictions_valid)
    r2 = r2_score(target_valid, predictions_valid)
    return print('RMSE:', rmse), print('MAE:', mae), print('R^2:', r2), \
    print('Средний запас сырья на валидационной выборке:', target_valid.mean()), \
    print('Средний запас предсказанного сырья:', predictions_valid.mean()), \
    print('Разница:', predictions_valid.mean()-target_valid.mean())

In [12]:
def predictions (features_train, target_train, features_valid):
    lr = LinearRegression()
    lr.fit(features_train, target_train)
    predictions_valid = lr.predict(features_valid)
    return predictions_valid

In [13]:
lr_1 = regression(features_train_1, target_train_1, features_valid_1, target_valid_1)
predictions_valid_1 = predictions(features_train_1, target_train_1, features_valid_1)
lr_1

RMSE: 37.5794217150813
MAE: 30.919600777151313
R^2: 0.27994321524487786
Средний запас сырья на валидационной выборке: 92.07859674082927
Средний запас предсказанного сырья: 92.59256778438035
Разница: 0.5139710435510807


(None, None, None, None, None, None)

Для первого региона обучена модель линейной регрессии. Показатель RMSE составляет 37.58 тыс. баррелей, MAE - 30.92 тыс. баррелей, средний запас предсказанного сырья - 92.59 тыс. баррелей.  
Таким образом, среднее отклонение предсказаний от фактических данных составило 30.92 тыс. баррелей. Стоит отметить, что разница между фактическим средним запасом сырья и предсказанным средним запасом и на валидационной выборке составила 514 баррелей.

In [14]:
lr_2 = regression(features_train_2, target_train_2, features_valid_2, target_valid_2)
predictions_valid_2 = predictions(features_train_2, target_train_2, features_valid_2)
lr_2

RMSE: 0.893099286775617
MAE: 0.7187662442124758
R^2: 0.9996233978805127
Средний запас сырья на валидационной выборке: 68.72313602435997
Средний запас предсказанного сырья: 68.728546895446
Разница: 0.005410871086027669


(None, None, None, None, None, None)

Для второго региона показатель RMSE составил 0.89 тыс. баррелей, MAE - 0.72 тыс. баррелей, средний запас предсказанного сырья - 68.73 тыс. баррелей.  
Разница между фактическим средним запасом сырья и предсказанным средним запасом и на валидационной выборке составила 5.41 баррель.

In [15]:
lr_3 = regression(features_train_3, target_train_3, features_valid_3, target_valid_3)
predictions_valid_3 = predictions(features_train_3, target_train_3, features_valid_3)
lr_3

RMSE: 40.02970873393434
MAE: 32.792652105481814
R^2: 0.20524758386040443
Средний запас сырья на валидационной выборке: 94.88423280885438
Средний запас предсказанного сырья: 94.96504596800489
Разница: 0.0808131591505088


(None, None, None, None, None, None)

Для третьего региона показатель RMSE составил 40.03 тыс. баррелей, MAE - 32.79 тыс. баррелей, средний запас предсказанного сырья - 94.97 тыс. баррелей.  
Разница между фактическим средним запасом сырья и предсказанным средним запасом и на валидационной выборке составила 80.81 баррель.

Таким образом, нами построены модели линейной регрессии для каждого региона. Значения отклонений предсказаний объема сырья и фактического объема в регионах различны.
Наименьшие отклонения получены во втором регионе, в котором показатель МАЕ (среднее обсолютное отклонение) составляет 718.76 баррелей нефти. В первом регионе МАЕ = 30.92 тыс. баррелей, в третьем - 32.79 тыс. баррелей.


## Подготовка к расчёту прибыли

Условия задачи:
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

In [16]:
BUDGET = 10_000_000_000
INCOME = 450_000
TOTAL_WELLS = 500
BEST_WELLS = 200

In [17]:
volume = BUDGET/INCOME/BEST_WELLS
volume

111.11111111111111

Для безубыточной разработки новой скважины объем сырья в ней должен составлять 111.12 тыс. баррелей.

In [18]:
print('Средний объем сырья в скважине первого региона', df_1['product'].mean())
print('Средний объем сырья в скважине второго региона', df_2['product'].mean())
print('Средний объем сырья в скважине третьего региона', df_3['product'].mean())

Средний объем сырья в скважине первого региона 92.50000000000001
Средний объем сырья в скважине второго региона 68.82500000000002
Средний объем сырья в скважине третьего региона 95.00000000000004


Средний объем сырья в скважинах каждого региона меньше, чем необходимый объем сырья для безубыточной разработки. Наименьший средний объем сырья наблюдается во втором регионе.

## Расчёт прибыли и рисков 

In [19]:
def revenue(target, predictions, count, income):
    pred_sorted = predictions.sort_values(ascending=False)
    selected = target[pred_sorted.index][:count]
    return (INCOME * selected.sum())-BUDGET

Напиcали функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели.

In [20]:
predictions_valid_1 = pd.Series(predictions_valid_1, index=target_valid_1.index)
predictions_valid_2 = pd.Series(predictions_valid_2, index=target_valid_2.index)
predictions_valid_3 = pd.Series(predictions_valid_3, index=target_valid_3.index)

In [21]:
print('Прибыль для первого региона:', revenue(target_valid_1, predictions_valid_1, BEST_WELLS, INCOME))
print('Прибыль для второго региона:', revenue(target_valid_2, predictions_valid_2, BEST_WELLS, INCOME))
print('Прибыль для третьего региона:', revenue(target_valid_3, predictions_valid_3, BEST_WELLS, INCOME))

Прибыль для первого региона: 3320826043.1398506
Прибыль для второго региона: 2415086696.681511
Прибыль для третьего региона: 2710349963.5998325


Прибыль для 200 скважин, выбранных на основе предсказанных данных составила:


1.   Для первого региона: 3.32 млрд.руб.;
2.   Для второго региона: 2.42 млрд.руб.;
3.   Для третьего региона: 2.71 млрд.руб.


In [22]:
state = np.random.RandomState(12345)
def bootstrap_revenue(target_valid, predictions_valid):     
  values = []
  for i in range(1000):
    target_subsample = target_valid.sample(TOTAL_WELLS, replace=True, random_state=state)
    pred_subsample = predictions_valid[target_subsample.index] 
    values.append(revenue(target_subsample, pred_subsample, BEST_WELLS, INCOME))
  return values

Создали функцию, в которой применили технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

In [23]:
revenue_1 = pd.Series(bootstrap_revenue(target_valid_1, predictions_valid_1))

In [24]:
revenue_2 = pd.Series(bootstrap_revenue(target_valid_2, predictions_valid_2))

In [25]:
revenue_3 = pd.Series(bootstrap_revenue(target_valid_3, predictions_valid_3))

Для каждого региона применили функцию.

In [26]:
print('Средняя прибыль для первого региона:', revenue_1.mean())
print('Средняя прибыль для второго региона:',revenue_2.mean())
print('Средняя прибыль для третьего региона:',revenue_3.mean())

Средняя прибыль для первого региона: 425938526.91059244
Средняя прибыль для второго региона: 518259493.6973249
Средняя прибыль для третьего региона: 420194005.3440501


Для первого региона средняя прибыль составляет 425.94 млн. руб., для второго - 518.26 млн. руб., для третьего - 420.19 млн. руб. 

In [27]:
print('95%-й доверительный интервал прибыли для первого региона: от', revenue_1.quantile(0.025),'до', revenue_1.quantile(0.975))
print('95%-й доверительный интервал прибыли для второго региона: от',revenue_2.quantile(0.025),'до', revenue_2.quantile(0.975))
print('95%-й доверительный интервал прибыли для третьего региона: от',revenue_3.quantile(0.025),'до', revenue_3.quantile(0.975))

95%-й доверительный интервал прибыли для первого региона: от -102090094.83793654 до 947976353.3583689
95%-й доверительный интервал прибыли для второго региона: от 128123231.43308444 до 953612982.0669085
95%-й доверительный интервал прибыли для третьего региона: от -115852609.16001143 до 989629939.8445739


С 95% вероятностью прибыль для отдельно взятого месторождения составит:


1.   Для первого региона: от -102.09 млн. руб. до 947.98 млн. руб.;
2.   Для второго региона: от 128.12 млн. руб. до 953.61 млн. руб.;
3.   Для третьего региона: от -115.85 млн. руб. до 989.63 млн. руб.

In [28]:
print('95%-й доверительный интервал для первого региона:', st.t.interval(0.95, len(revenue_1)-1, revenue_1.mean(), revenue_1.sem()))
print('95%-й доверительный интервал для второго региона:', st.t.interval(0.95, len(revenue_2)-1, revenue_2.mean(), revenue_2.sem()))
print('95%-й доверительный интервал для третьего региона:', st.t.interval(0.95, len(revenue_3)-1, revenue_3.mean(), revenue_3.sem()))

95%-й доверительный интервал для первого региона: (408732207.06869864, 443144846.7524862)
95%-й доверительный интервал для второго региона: (505249881.5766217, 531269105.81802803)
95%-й доверительный интервал для третьего региона: (402528703.65036154, 437859307.0377387)


С 95% вероятностью средняя прибыль составляет:


1.   Для первого региона: от 408.73 млн. руб. до 443.14 млн. руб.;
2.   Для второго региона: от 505.25 млн. руб. до 531.13 млн. руб.;
3.   Для третьего региона: от 402.53 млн. руб. до 437.86 млн. руб.




In [29]:
print('Риск убытков для первого региона: {:.2%}'. format((revenue_1 < 0).mean()))
print('Риск убытков для второго региона: {:.2%}'. format((revenue_2 < 0).mean()))
print('Риск убытков для третьего региона: {:.2%}'. format((revenue_3 < 0).mean()))

Риск убытков для первого региона: 6.00%
Риск убытков для второго региона: 0.30%
Риск убытков для третьего региона: 6.20%


Риск убытков (соотношение отрицательного показателя прибыли к количеству скважин) составил:


1.   Для первого региона 6%
2.   Для второго региона 0.3%
3.   Для третьего региона 6.2%



## Вывод

Таким образом, лучшим регионам для разработки скважин является второй регион, так как с 95% вероятностью полученная прибыль от добычи нефти в нем составит от от 505.25 млн. руб. до 531.13 млн. руб., что является максимальным значением среди всех регионов, при этом риск убытков является минимальным и составляет 0.3%. Кроме того, модель линейной регрессии для второго региона является наилучшей по метрикам RMSE, MAE и R^2.